# 외부와 상호작용 하면서 자율적으로 행동하는 Agents
## 언어 모델에 도구를 부여할 수 있다.
- Agents 모듈을 사용해 다양한 작업에 맞는 도구를 선택해 실행할 수 있다.
- 다음과 같은 두 가지 하위 모듈이 있다.
    - Tool
    - Agent
### 말 그대로 다양한 **도구**가 되는 Tool
- Tool은 언어 모델만으로 할 수 없는 일을 할 수 있게 해주는 모듈
- 계산, 검색 등 다양한 종류가 있어 목적에 따라 선택
    - LLMMath
        - 언어 모델이 취약한 계산
    - Requests
        - 지정된 URL로 요청 보낼 수 있게 해줌, 웹사이트의 정보 취득, 인터넷에 공개된 API 사용 등
    - File System Tools
        - PC내 파일에 접근, 지정된 경로에서 파일 읽고 쓰기 가능, 실행 결과 저장 등
    - SerpApi
        - 구글이나 야후 검색을 API로 하는 SerpApi 라는 웹 서비스와 연동동

### 명령에 따라 자율적으로 처리를 수행하는 Agent
- Agent는 Tool을 선택 하고 다음 단계의 처리를 수행하는 주체
- ReAct 기법 
    1. 사용자로 부터 작업을 받는다
    2. 준비된 Tool 중에서 어떤 Tool을 사용할지, 어떤 정보를 입력 할 지 결정
    3. Tool를 사용해 결과를 얻는다
    4. 3에서 얻은 결과를 통해 과업이 달성되고 있는지 확인한다
    5. 에이전트가 작업을 완료했다고 판단할 수 있을 때까지 2~4를 반복한다



### Agent 시스템 개요 : [소스](https://lilianweng.github.io/posts/2023-06-23-agent/)
- LLM 기반 자율 에이전트 시스템에서 LLM은 에이전트의 두뇌 역할을 하며 다음과 같은 몇 가지 주요 구성 요소로 보완됩니다.
- 계획
    - 하위 목표 및 분해: 에이전트는 대규모 작업을 더 작고 관리 가능한 하위 목표로 나누어 복잡한 작업을 효율적으로 처리할 수 있도록 합니다.
    - 반성 및 개선: 에이전트는 과거 행동에 대한 자기비판 및 반성을 할 수 있으며, 실수로부터 학습하고 이를 향후 단계에 맞게 다듬어 최종 결과의 품질을 향상시킬 수 있습니다.
- 메모리
    - 단기 기억: 모든 상황 내 학습( Prompt Engineering 참조 )은 모델의 단기 기억을 활용하여 학습하는 것으로 간주합니다.
    - 장기 기억: 이는 대개 외부 벡터 저장소와 빠른 검색을 활용하여 장기간에 걸쳐 (무한) 정보를 유지하고 회상할 수 있는 기능을 에이전트에 제공합니다.
- tools 사용
    - 에이전트는 현재 정보, 코드 실행 기능, 독점 정보 소스에 대한 액세스 등을 포함하여 모델 가중치에서 누락된 추가 정보(사전 학습 후 변경하기 어려운 경우가 많음)를 위해 외부 API를 호출하는 방법을 학습합니다.

 
<img src="https://lilianweng.github.io/posts/2023-06-23-agent/agent-overview.png">

## 주어진 URL에서 정보 얻게 하기
- URL에 엑세스 할 수 있는 Tool을 가진 Agent를 만들어 보자

In [106]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import ChatOpenAI

# 모델 초기화
chat = ChatOpenAI(
    temperature=0, # 응답 다양성 설정, 0~2, 기본값은 0.7
    model='gpt-3.5-turbo'
)

tools = load_tools( # 렝체인에 준비된 Tool 로드
    [
        "requests_get", # 특정 URL의 결과를 얻는 Tool, requests과 requests_get는 보안상 문제가 있음
    ],
    allow_dangerous_tools=True # 이 옵션을 주어야만 사용 가능
)

agent = initialize_agent( # Agent 초기화
    tools=tools, # 사용할 tools 리스트
    llm=chat, # 언어모델 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, # ReAct 방식으로 작동하게 설정
    verbose=True # 로그 표시
)

In [108]:
result = agent.run("""아래 URL에 접속해 도쿄의 날씨를 검색해 한국어로 말하세요.
https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json
""")



> Entering new AgentExecutor chain...
Question: Access the URL provided and search for the weather in Tokyo. Translate it into Korean.
Thought: I should use the requests_get tool to retrieve the weather information from the URL.
Action:
```
{
  "action": "requests_get",
  "action_input": "https://www.jma.go.jp/bosai/forecast/data/overview_forecast/130000.json"
}
```

Observation: {"publishingOffice":"気象庁","reportDatetime":"2024-06-08T04:38:00+09:00","targetArea":"東京都","headlineText":"","text":"　本州付近は緩やかに高気圧に覆われていますが、湿った空気の影響を受けています。\n\n　東京地方は、晴れや曇りとなっています。\n\n　８日は、緩やかに高気圧に覆われますが、湿った空気の影響を受ける見込みです。このため、晴れ朝晩曇りとなるでしょう。\n\n　９日は、引き続き、緩やかに高気圧に覆われますが、低気圧や湿った空気の影響を受ける見込みです。このため、曇り昼前まで時々晴れとなるでしょう。\n\n【関東甲信地方】\n　関東甲信地方は、晴れや曇りで伊豆諸島では雨の降っている所があります。\n\n　８日は、緩やかに高気圧に覆われますが、湿った空気の影響を受ける見込みです。このため、晴れや曇りで雨の降る所があるでしょう。\n\n　９日は、引き続き、緩やかに高気圧に覆われますが、低気圧や湿った空気の影響を受ける見込みです。このため、曇りや晴れで、伊豆諸島では夜は雨が降るでしょう。\n\n　関東地方と伊豆諸島の海上では、うねりを伴い８日は波が高く、９日は波がやや高い見込みです。"}
Thought:I will need to translate the weather informati

In [109]:
print(f'실행 결과 : {result}')

실행 결과 : 도쿄 지방은 맑고 흐려져 있습니다. 8일은 완만하게 고기압으로 덮여 있습니다만, 습한 공기의 영향을 받는 보기입니다. 이 때문에 맑은 아침 저녁 흐림이 될 것입니다. 9일은, 계속해, 완만하게 고기압에 덮여 있습니다만, 저기압이나 습한 공기의 영향을 받는 견후입니다. 이 때문에, 흐린 낮전까지 때때로 맑게 될 것입니다. 【칸토 고신 지방】 칸토 고신 지방은 맑고 흐린 이즈 제도에서는 비가 내리는 곳이 있습니다. 8일은 완만하게 고기압으로 덮여 있습니다만, 습한 공기의 영향을 받는 보기입니다. 이 때문에 맑고 흐린 비가 내리는 곳이 있을 것입니다. 9일은, 계속해, 완만하게 고기압에 덮여 있습니다만, 저기압이나 습한 공기의 영향을 받는 견후입니다. 이 때문에, 흐림이나 가끔 맑고, 이즈 제도에서는 밤에는 비가 내릴 것입니다. 칸토 지방과 이즈 제도의 해상에서, 파도를 수반해 8일은 파도가 높고, 9일은 파도가 약간 높은 견후입니다.


#### Agent의 종류
- Agent 는 Tool을 사용 특정 테스크를 달성 한다. Agent를 조작하는 방법은 다양하며 이에 따라 툴의 조작 방법도 다르다.
    - [link](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_types.AgentType.html)
    - AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION
        - 대화 형식의 상호 작용이 특징, Chat models 모듈과 ReAct 방법을 사용해 작동
    - AgentType.ZERO_SHOT_REACT_DESCRIPTION
        - LLMs 모듈 사용을 전제로 하는 에이전트
    - AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION
        - 복수개의 inputs 을 가진 툴을 사용

# Tool을 추가해 Agent가 할 수 있는 일 늘리기

## Agent가 할 수 있는 것은 전달하는 Tool에 따라 달라진다
- 파일 쓰기와 인터넷 검색을 할 수 있는 Tool을 추가 해보자
- SerpApi 라는 서비스를 사용 인터넷에서 정보를 가져 온다.
    - 구글 등 검색 엔진의 검색 결과를 가져올 수 있는 서비스

## 환경 변수에 SerpApi의 API Key 설정하기
- 각 OS에 맞게 API를 환경 변수에 등록해 준다.

## google-search-results 설치하기
- SerpApi에서 구글 검색을 하려면 google-search-results 라는 파이썬 페키지 필요
- pip install google-search-results 

In [19]:
# !pip install google-search-results

In [113]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain.tools.file_management import WriteFileTool # 파일 쓰기를 할 수 있는 Tool

chat = ChatOpenAI(
    temperature=0,
    model='gpt-3.5-turbo'
)

tools = load_tools(
    [
        'requests_get',
        'serpapi'
    ],
    allow_dangerous_tools =True,
    llm=chat
)

tools.append(WriteFileTool(
    root_dir="./"
))

agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, # WriteFIleTool과 같이 복수개의 입력을 받을 수 있는 Tool 사용 가능,
    verbose=True
)

In [114]:
result = agent.run("경주시의 특산품을 검색해 result.txt 파일에 한국어로 저장하세요.")



> Entering new AgentExecutor chain...
Thought: I can use a search engine to find the specialty products of Gyeongju City and then save the information in Korean to a file.

Action:
```
{
  "action": "Search",
  "action_input": "특산품 경주시"
}
```
Observation: ['감포특산물 type: 상점.', '감포특산물 entity_type: local_nav.', '감포특산물 kgmid: /g/11f64dh3rn.', '감포특산물 place_id: ChIJ6yZGuJalZzURa8N7WR5ie0A.', '감포특산물 phone: +82 54-774-6889.', '감포특산물 address: South Korea, Gyeongsangbuk-do, Gyeongju-si, Gampo-eup, Gampo-ri, 484-3 감포읍 감포리 484-3번지 경주시 경상북도 KR 38125.', '감포특산물 raw_hours: Closed ⋅ Opens 10 AMSaturday10 AM–8 PMSunday10 AM–8 PMMonday10 AM–8 PMTuesday10 AM–8 PMWednesday10 AM–8 PMThursday10 AM–8 PMFriday10 AM–8 PMSuggest new hoursUnable to add this file. Please check that it is a valid photo..', '경주시 대표 과채류로 형상강 하류의 유기물이 풍부하고 물빠짐이 좋은 토양환경과 평야지에서 많은 일조량을 받고 자라 과육이 풍부합니다. 당도가 높은 동양계 품종으로 ...', '교동법주 정보 · 전화 054-772-2051 · 주소 교촌안길 19-21 · 가격 교동법주900ml(종이상자) 35,000원 / 교동법주900ml(나무상자) 40,000원 ...', '카테고리 이동 

### ReAct 기법과 OpenAI Function Calling
- OpenAI의 언어모델만 사용한다면,
    - CHAT_ZERO_SHOT_REACT_DESCRIPTION 대신에 OPENAI_FUNCTIONS (단일 입력)
    - STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION 대신에 OPENAI_MULTI_FUNCTIONS (복수 입력)
- 을 사용해도 유사한 결과가 나온다.

#### Agent Type
- Agent의 종류는 여러 종류입니다. 목적하는 바는 유사하지만 동작하는 프롬프트를 주는 디테일이 다릅니다.
    - https://langchain-langchain.vercel.app/docs/modules/agents/agent_types/
    - https://platform.openai.com/docs/guides/function-calling

| Agent Type            | Intended Model Type | Supports Chat History | Supports Multi-Input Tools | Supports Parallel Function Calling | Required Model Params | When to Use |
|-----------------------|---------------------|------------------------|---------------------------|-----------------------------------|-----------------------|-------------|
| OpenAI 도구           | 채팅                | ✅                      | ✅                          | ✅                                 | tools                 | 최신 OpenAI 모델 (1106 이후)을 사용하는 경우 |
| ~~OpenAI 함수~~           | 채팅                | ✅                      | ✅                          |                                   | functions             | OpenAI 모델을 사용하거나, 함수 호출을 위해 미세 조정된 오픈 소스 모델을 사용하고 OpenAI와 동일한 함수 매개변수를 노출하는 경우 |
| XML                   | LLM                 | ✅                      |                            |                                   |                       | Anthropic 모델이나 XML에 능숙한 다른 모델을 사용하는 경우 |
| Structured Chat         | 채팅                | ✅                      | ✅                          |                                   |                       | 다중 입력 도구를 지원해야 하는 경우 |
| JSON Chat             | 채팅                | ✅                      |                            |                                   |                       | JSON에 능숙한 모델을 사용하는 경우 |
| ReAct                 | LLM                 | ✅                      |                            |                                   |                       | 간단한 모델을 사용하는 경우 |
| Self Ask With Search      | LLM                 |                        |                            |                                   |                       | 간단한 모델을 사용하고 하나의 검색 도구만 있는 경우 |

In [24]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain.tools.file_management import WriteFileTool # 파일 쓰기를 할 수 있는 Tool

chat = ChatOpenAI(
    temperature=0,
    model='gpt-3.5-turbo'
)

tools = load_tools(
    [
        'requests_get',
        'serpapi'
    ],
    allow_dangerous_tools =True,
    llm=chat
)

tools.append(WriteFileTool(
    root_dir="./"
))

agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.OPENAI_MULTI_FUNCTIONS, # WriteFIleTool과 같이 복수개의 입력을 받을 수 있는 OpenAI의 Function callings
    verbose=True
)

In [25]:
result = agent.run("경주시의 특산품을 검색해 result.txt 파일에 한국어로 저장하세요.")



> Entering new AgentExecutor chain...

Invoking: `Search` with `{'tool_input': 'Specialty products of Gyeongju'}`


['Bulgogi (broiled meat) · Hwangnam-ppang (Gyeongju bread) · Garlic chicken · Chueo-tang (loach soup) · Hoe (sashimi) · Sundubu-jjigae (soft tofu stew) · Haejang-guk ( ...', 'Gift & Specialty Shops in Gyeongju · Good for Kids · Good for Big Groups · Free Entry · Adventurous · Budget-friendly · Good for a Rainy Day · Hidden Gems · Good for ...', 'Local Specialties · Silla Earthenware · Silla Gold Crown and Silla Silver Cup · Quilted Clothes · HWANG NAM BBANG(cake) · Sticky Barley Bread.', "There are many items that can be easily purchased and give a gift to your family and friends as 'Commemoration of travel in Gyeongju' souvenirs such as wood ...", 'Travel back to the Silla era with the items defining the Gyeongju National Museum · 1. Gold Crown · 2. Divine Bell of King Seongdeok · 3. Bhaisajyaguru Buddha · 4.', 'Gyeongju was the capital of the ancient kingdom of Silla 

# Tool을 직접 제작해 기능 확장하기
## Tool을 직접 만들어서 할 수 있는 일의 폭을 더욱 넓힌다
- Agent 모듈에서는 사용자가 직접 Tool을 쉽게 생성 가능하다.
- Tool의 구성 요소와 Agent가 Tools 선택 하는 메커니즘을 우선 살펴 보자
- Tool은 다음과 같은 세 가시 요소로 구성되어 있다.
    - 이름 : Tool을 식별하기 위한 이름 예) Calculator, requests_get
    - 기능 설명 : Tool이 무엇을 하는지에 대한 간략한 설명, Agent가 Tool을 어떻게 사용할지 판단 할 수 있는 자료
    - 실행 함수 : Tool이 실제 작동할 때 처리

- 특정 숫자 이상의 임의의 숫자를 생성하는 Tool 만들어 보자

In [115]:
import random
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import ChatOpenAI
from langchain.tools import WriteFileTool

chat = ChatOpenAI()

tools= []

tools.append(WriteFileTool(root_dir='./'))

def min_limit_random_number(min_number):
    return random.randint(int(min_number), 1000000)

tools.append( # 도구 추가
    Tool(
        name="Random", # 도구 이름 
        description="특정 최솟값 이상의 임의의 숫자를 생성할 수 있습니다", # 도구 설명
        func=min_limit_random_number # 도구가 실행될 떄 호출되는 함수
    )
)

agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

result = agent.run("10 이상의 난수를 생성해 random.txt 파일에 저장하세요.")

print(f'실행 결과 : {result}')



> Entering new AgentExecutor chain...
Thought: I should generate a random number greater than or equal to 10 and save it to a file.

Action:
```
{
  "action": "Random",
  "action_input": {
    "tool_input": "10"
  }
}
```
Observation: 543776
Thought:I have generated a random number greater than or equal to 10. Now, I will save it to a file.

Action:
```
{
  "action": "write_file",
  "action_input": {
    "file_path": "random.txt",
    "text": "543776",
    "append": false
  }
}
``` 

Observation: File written successfully to random.txt.
Thought:I have generated a random number greater than or equal to 10 and saved it to the "random.txt" file successfully. 
Action:
```
{
  "action": "Final Answer",
  "action_input": "Random number greater than or equal to 10 has been saved to random.txt."
}
```

> Finished chain.
실행 결과 : Random number greater than or equal to 10 has been saved to random.txt.


# Retrievers를 사용해 문장을 검색하는 Tool 만들기
## Retrievers는 Tool로 변환할 수 있다.
- Agent는 어떤 Tool을 사용할 지 스스로 결정하고, Tool을 사용해 작업 수행 한다.
- SerpApi를 사용해 구글에서 검색을 하고, Retrievers를 사용해 위키백과나 구축한 데이터 베이스에서 검색을 한다.

In [29]:
# !pip install wikipedia

In [118]:
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.agents.agent_toolkits import create_retriever_tool  #← create_retriever_tool을 가져오기
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import WikipediaRetriever #←WikipediaRetriever를 가져오기
from langchain.tools import WriteFileTool

chat = ChatOpenAI()

tools = [] 

tools.append(WriteFileTool( 
    root_dir="./"
))

retriever = WikipediaRetriever( #←WikipediaRetriever를 초기화
    lang="ko", #←언어를 한국어로 설정
    doc_content_chars_max=1500,  #←글의 최대 글자 수를 500자로 설정
    top_k_results=1 #←검색 결과 중 상위 1건을 가져옴
)

tools.append(
    create_retriever_tool(  #←Retrievers를 사용하는 Tool을 생성
        name="WikipediaRetriever",  #←Tool 이름
        description="받은 단어에 대한 Wikipedia 기사를 검색할 수 있다",  #←Tool 설명
        retriever=retriever,  #←Retrievers를 지정
    )
)

agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  
    verbose=True
)

result = agent.run("스카치 위스키에 대해 Wikipedia에서 찾아보고 그 개요를 요약하여 한국어로 result2.txt 파일에 저장하세요.")

print(f"실행 결과: {result}")



> Entering new AgentExecutor chain...
Thought: I will use the WikipediaRetriever tool to search for information about "스카치 위스키" and then summarize the overview in Korean before saving it to a file.

Action:
```
{
  "action": "WikipediaRetriever",
  "action_input": {
    "query": "스카치 위스키"
  }
}
```
Observation: 스카치 위스키(Scotch whisky)는 스코틀랜드에서 제조되는 위스키이다. 영국에서는 "위스키"(whisky)라고 하면, 특별한 언급이 없는 한 스카치 위스키를 가리킨다. 미국에서는 짧게 줄여 "스카치"라고도 불린다.


== 역사 ==

“To Friar John Cor, by order of the King, to make aqua vitae VIII bolls of malt.” — Exchequer Rolls 1494–95, vol x, p. 487.
위스키는 영국 스코틀랜드 지방에서 4~5세기 무렵에 수도승들이 가지고 들어왔다고 추정되는 증류주 제조술을 바탕으로 제조되어 왔다. 1644년에는 위스키 제조에 세금이 붙기 시작하였는데, 이는 전국적으로 은밀한 위스키 제조를 성행하게 만들었고, 급기야는 1780년 무렵 합법 증류소는 8곳인데 반해 불법 증류소는 400여곳이 될 정도로 불법 위스키 제조가 성행하였다. 이에 1823년, 영국 의회는 "소비세법"을 강화하여 면허를 가지고 있는 합법적 증류소에 대한 규제는 푸는 대신, 불법 증류소의 운영을 어렵게 함으로써 불법 위스키 제조를 단절시켰다.
1831년, 위스키의 톡 쏘는 맛이 줄어들면서 부드러운 맛이 나오는 "코피"(Coffey) 혹은 "패튼트" 스틸이라고 부르는 제조법의 등장 및 1880년, 포도나무뿌리진디라는 해충으로 인해 프랑스의 코냑과 포도

# 문맥에 맞게 답변하는 에이전트 만들기
## 대화 기록을 보관하는 에이전트 생성하기

In [119]:
from langchain.agents import AgentType, initialize_agent
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory  #←ConversationBufferMemory 가져오기
from langchain.retrievers import WikipediaRetriever

chat = ChatOpenAI()

tools = []

retriever = WikipediaRetriever(
    lang="ko",
    doc_content_chars_max=500,
    top_k_results=1
)

tools.append(
    create_retriever_tool(  #←Retrievers를 사용하는 Tool을 생성
        name="WikipediaRetriever",  #←Tool 이름
        description="받은 단어에 대한 Wikipedia 기사를 검색할 수 있다",  #←Tool 설명
        retriever=retriever,  #←Retrievers를 지정
    )
)

memory = ConversationBufferMemory(  #←ConversationBufferMemory를 초기화
    memory_key="chat_history",  #←메모리 키를 설정
    return_messages=True  #←메시지를 반환하도록 설정
)

agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,  #←Agent의 유형을 대화형으로 변경
    memory=memory,  #←Memory를 지정
    verbose=True
)

result = agent.run("스카치 위스키에 대해 Wikipedia에서 찾아보고 그 개요를 한국어로 요약 정리하세요.") #←Wikipedia에서 찾아보라고 지시
print(f"1차 실행 결과: {result}") #←실행 결과를 표시



> Entering new AgentExecutor chain...
```json
{
    "action": "WikipediaRetriever",
    "action_input": "스카치 위스키"
}
```
Observation: 스카치 위스키(Scotch whisky)는 스코틀랜드에서 제조되는 위스키이다. 영국에서는 "위스키"(whisky)라고 하면, 특별한 언급이 없는 한 스카치 위스키를 가리킨다. 미국에서는 짧게 줄여 "스카치"라고도 불린다.


== 역사 ==

“To Friar John Cor, by order of the King, to make aqua vitae VIII bolls of malt.” — Exchequer Rolls 1494–95, vol x, p. 487.
위스키는 영국 스코틀랜드 지방에서 4~5세기 무렵에 수도승들이 가지고 들어왔다고 추정되는 증류주 제조술을 바탕으로 제조되어 왔다. 1644년에는 위스키 제조에 세금이 붙기 시작하였는데, 이는 전국적으로 은밀한 위스키 제조를 성행하게 만들었고, 급기야는 1780년 무렵 합법 증류소는 8곳인데 반해 불법 증류소는 400여곳이 될 정도로 불법 위스키 제조가 성행하였다. 이에 1823년, 영국 의회는 "소비세법"을 강화하여 면허를 가
Thought:```json
{
    "action": "Final Answer",
    "action_input": "스카치 위스키(Scotch whisky)는 스코틀랜드에서 제조되는 위스키이다. 영국에서는 '위스키'(whisky)라고 하면, 특별한 언급이 없는 한 스카치 위스키를 가리킨다. 미국에서는 짧게 줄여 '스카치'라고도 불린다. 위스키는 영국 스코틀랜드 지방에서 4~5세기 무렵에 수도승들이 가지고 들어왔다고 추정되는 증류주 제조술을 바탕으로 제조되어 왔다. 1644년에는 위스키 제조에 세금이 붙기 시작하였는데, 이는 전국적으로 은밀한 위스키 제조를 성행하게 만들었고, 급기야는 1780년 무렵 합법 증류소는 8곳인데 반해 불법 증류

In [121]:
result_2 = agent.run("이전 지시를 다시 한번 실행하세요.") #←이전 지시를 다시 실행하도록 지시
print(f"2차 실행 결과: {result_2}") #←실행 결과를 표시



> Entering new AgentExecutor chain...
```json
{
    "action": "WikipediaRetriever",
    "action_input": "스카치 위스키"
}
```
Observation: 스카치 위스키(Scotch whisky)는 스코틀랜드에서 제조되는 위스키이다. 영국에서는 "위스키"(whisky)라고 하면, 특별한 언급이 없는 한 스카치 위스키를 가리킨다. 미국에서는 짧게 줄여 "스카치"라고도 불린다.


== 역사 ==

“To Friar John Cor, by order of the King, to make aqua vitae VIII bolls of malt.” — Exchequer Rolls 1494–95, vol x, p. 487.
위스키는 영국 스코틀랜드 지방에서 4~5세기 무렵에 수도승들이 가지고 들어왔다고 추정되는 증류주 제조술을 바탕으로 제조되어 왔다. 1644년에는 위스키 제조에 세금이 붙기 시작하였는데, 이는 전국적으로 은밀한 위스키 제조를 성행하게 만들었고, 급기야는 1780년 무렵 합법 증류소는 8곳인데 반해 불법 증류소는 400여곳이 될 정도로 불법 위스키 제조가 성행하였다. 이에 1823년, 영국 의회는 "소비세법"을 강화하여 면허를 가
Thought:```json
{
    "action": "Final Answer",
    "action_input": "스카치 위스키는 스코틀랜드에서 제조되는 위스키로, 영국에서는 '위스키'라고 하면 대부분 스카치 위스키를 가리킨다. 미국에서는 '스카치'로도 불린다. 위스키는 수도승들이 증류주 제조술을 가져와서 제조되기 시작했으며, 1644년에는 위스키 제조에 세금이 붙기 시작했다. 불법 위스키 제조가 흔했지만 1823년에 영국 의회가 소비세법을 강화하여 합법적인 제조소를 지정했다."
}
```

> Finished chain.
2차 실행 결과: 스카치 위스키는 스코틀랜드에서 제조되는 위스키로, 영국에서는 '위스키'라고 하면 대부분 스카치 